In [2]:
from packaging import version
import sklearn
import tensorflow as tf

- 텐서플로 데이터 API는 대규모 데이터셋을 효율적으로 로드하고 전처리할 수 있도록 한다.
- 텐서플로가 멀티스레딩, 큐, 배치, 프리페치 같은 상세한 사항을 모두 대신 처리해준다.
- 또한 데이터 API는 tf.keras와도 잘 동작한다.

이 장에서는 데이터 API, TFRecord 포맷을 다루고 사용자 정의 전처리 층을 만드는 방법과 표준 케라스 전처리 층을 사용하는 법을 다룬다.

# 1. 데이터 API
- from_tensor_slices() 함수는 텐서를 받아 X의 각 원소가 아이템으로 표현되는 tf.data.Dataset을 만든다.<br>
    - 즉 데이터셋은 0,1,2,,9에 해당하는 10개의 아이템을 가진다.

In [3]:
X = tf.range(10) # 데이터 텐서
dataset = tf.data.Dataset.from_tensor_slices(X) # 메모리 전체에서 데이터셋 생성
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [5]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [23]:
# make dict
X_neated = {"a": ([1,2,3],[4,5,6]), "b":[7,8,9]}
dataset = tf.data.Dataset.from_tensor_slices(X_neated)
for item in dataset:
    print(item)

{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=4>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=7>}
{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=int32, numpy=5>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=8>}
{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=3>, <tf.Tensor: shape=(), dtype=int32, numpy=6>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=9>}


## 1.1 연쇄변환
데이터셋이 준비되면 변환 메서드를 호출하여 여러 종류의 변환을 수행할 수 있다.
- repeat(x) 메서드: 반복횟수. 원본 데이터셋의 아이템을 x차례 반복하는 새로운 데이터셋 반환
- batch(x) 메서드: 배치크기

In [62]:
dataset = tf.data.Dataset.from_tensor_slices(tf.range(10))
dataset = dataset.repeat(3).batch(7)
dataset_dr = dataset.repeat(3).batch(7, drop_remainder=True) # drop_remainder=True로 하면 남은 [8 9]는 drop
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


map으로 데이터 변환<br>
- num_parallel_calls 매개변수로 여러 스레드로 나누어 속도를 높일 수 있음.
- map() 메서드에 전달하는 함수는 텐서플로 함수로 변환 가능해야 함.

In [63]:
# 각 아이템에 *2 해주기
dataset = dataset.map(lambda x:x * 2) # x는 배치
for item in dataset:
    print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
tf.Tensor([16 18], shape=(2,), dtype=int32)


In [67]:
# reduce_sum : 텐서의 모든 성분의 총합을 계산하는 함수
dataset = dataset.filter(lambda x: tf.reduce_sum(x) > 50)
for item in dataset:
    print(item)

tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)


In [74]:
# 위의 dataset 중에서 2개만 보고싶을 때
for item in dataset.take(2):
    print(item)

tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)


## 2. Shuffling the data
경사 하강법은 훈련셋에 있는 샘플이 독립적이고 동일한 분포일 때 최고 성능을 발휘한다. shuffle() 메서드로 간단하게 샘플을 섞어 데이터 분포를 적절하게 만들 수 있다.<br>
- shuffle() 메서드<br>
  [동작순서]<br>
    1. 먼저 원본 데이터셋의 처음 아이템은 buffer_size 개수만큼 추출하여 버퍼에 채운다.
    2. 그다음 새로운 아이템이 요청되면 이 버퍼에서 비워진 버퍼를 채운다.
    3. 원본 데이터셋의 모든 아이템이 사용될 때까지 반복된다.
    4. 그다음엔 버퍼가 비워질 때까지 계속라여 랜덤하게 아이쳄을 반환한다.
  - 이 메서드를 사용하려면 버퍼 크기를 지정해야 한다. 메모리를 넘지 않는 선에서 버퍼 크기를 충분히 크게 하는 것이 중요하다.<br>
 
⬇︎ 정수 0에서 9까지 세 번 반복된 데이터셋을 만든 다음, 버퍼 크기 4, 랜덤 시드 42를 사용하여 셔플링하고, 배치 크기 7로 나누어 출력

In [87]:
dataset = tf.data.Dataset.range(10).repeat(2)
dataset = dataset.shuffle(buffer_size=4, seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([1 4 2 3 5 0 6], shape=(7,), dtype=int64)
tf.Tensor([9 8 2 0 3 1 4], shape=(7,), dtype=int64)
tf.Tensor([5 7 9 6 7 8], shape=(6,), dtype=int64)


❗️셔플된 데이터셋이 repeat() 메서드를 호출하면 기본적으로 반복마다 새로운 순서를 생성한다. 하지만 반복마다 동일한 순서를 사용해야 한다면 repeat() 메서드에 reshuffle_each_iteration=False를 지정해야 한다.